This is a temporary notebook

In [1]:
import pandas as pd

In [2]:
# The below code gives a huge error due to encoding issue
# kenta and I have struggled with this for at least an hour...
# df = pd.read_csv("forpractice.csv")

Here we get a huge error due to unicode-decoding error without specifying a decoder in read_csv. So we need to figure out at 0x93 position which decoder we need.

In [3]:
df = pd.read_csv("../data/15kenoku20180213.csv", encoding='cp932', skiprows= [0,1,2])

In [4]:
name = pd.read_csv("../data/15kenoku20180213.csv", encoding="cp932", nrows=1, header=None)

In [5]:
name.iat[0,0] # this is the name of the individual

'田中敬一郎'

In [6]:
df.head()

,時刻,活動強度,運動種別
0,0:00:00,0.0,計測なし
1,0:01:00,0.0,計測なし
2,0:02:00,0.0,計測なし
3,0:03:00,0.0,計測なし
4,0:04:00,0.0,計測なし


It looks like we did a pretty good job. I don't really understand what the columns are saying so... we should probably change the column names.

In [7]:
df.columns = ["Time","Mets","random"]

Let's also remove the 3rd column because we don't need it

In [8]:
df.drop('random', axis=1, inplace=True)
df.head()

,Time,Mets
0,0:00:00,0.0
1,0:01:00,0.0
2,0:02:00,0.0
3,0:03:00,0.0
4,0:04:00,0.0


Later on, we need to merge in 7 days worth of data for each of the individuals. Meaning there will be 7 csv files for each individuals and there are 20 individuals. Technically there will be 140 csv files in the path. However, for now we need to focus on differentiating the time. Since the Time variable only has the `HH:MM:SS` information, we need to add it such that it is `YYYYMMDD HH:MM:SS`.

First load the filenames into a list

In [9]:
import glob
path = r'../data' # specify the path of the data files
allFiles = glob.glob(path + '/*.csv') # this creates a list of all the files in the folder with /*.csv extension
allFiles

['../data/15ericdong20180213.csv',
 '../data/15ericdong20180212.csv',
 '../data/forpractice20180213.csv',
 '../data/forpractice20180212.csv',
 '../data/15kenoku20180211.csv',
 '../data/forpractice-Copy5.csv',
 '../data/15ericdong20180211.csv',
 '../data/forpractice.csv',
 '../data/forpractice-Copy4.csv',
 '../data/15ericdong20180210.csv',
 '../data/15kenoku20180213.csv',
 '../data/forpractice20180211.csv',
 '../data/forpractice20180210.csv',
 '../data/15kenoku20180210.csv',
 '../data/15kenoku20180212.csv']

In this example case we have random files but the files we're interested in are those that start wit ha number then a persons name then the date (e.g. 15kenoku20180210.csv). We need to extractthe date from the filename.

In [10]:
import re
re.search('../data/\d*'+'kenoku'+'(\d+).csv', allFiles[10]).group(1)

'20180213'

The above code uses regexp to extract the csv file from the 11th item in the list which is `'../data/15kenoku20180213.csv'`
Now we need to append that data onto the Time column

In [11]:
(pd.Series([re.search('../data/\d*'+'kenoku'+'(\d+).csv', allFiles[10]).group(1)]*df.shape[0]) + " " + df.Time).head()

0    20180213 0:00:00
1    20180213 0:01:00
2    20180213 0:02:00
3    20180213 0:03:00
4    20180213 0:04:00
dtype: object

In [12]:
df["Time"] = (pd.Series([re.search('../data/\d*'+'kenoku'+'(\d+).csv', allFiles[10]).group(1)]*df.shape[0]) + " " + df.Time)

In [13]:
df.head()

,Time,Mets
0,20180213 0:00:00,0.0
1,20180213 0:01:00,0.0
2,20180213 0:02:00,0.0
3,20180213 0:03:00,0.0
4,20180213 0:04:00,0.0


Let's convert the time column to datetime type

First create a new column that will convert the charactercolumn `Time` into a timestamp column

In [14]:
df['Time'] =  pd.to_datetime(df['Time'], format="%Y%m%d %H:%M:%S")

In [15]:
df.head()

,Time,Mets
0,2018-02-13 00:00:00,0.0
1,2018-02-13 00:01:00,0.0
2,2018-02-13 00:02:00,0.0
3,2018-02-13 00:03:00,0.0
4,2018-02-13 00:04:00,0.0


Now need to create a column that will subtract all time from the beginning of the day

In [16]:
(df.Time - pd.DataFrame({"FTime": [df.Time.tolist()[0]]*df.shape[0]}).FTime).astype('timedelta64[m]').head()

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

In [17]:
df['Timedif'] = (df.Time - pd.DataFrame({"FTime": [df.Time.tolist()[0]]*df.shape[0]}).FTime).astype('timedelta64[m]')

In [18]:
df.tail()

,Time,Mets,Timedif
1435,2018-02-13 23:55:00,0.0,1435.0
1436,2018-02-13 23:56:00,0.0,1436.0
1437,2018-02-13 23:57:00,0.0,1437.0
1438,2018-02-13 23:58:00,0.0,1438.0
1439,2018-02-13 23:59:00,0.0,1439.0


Finally we have a column that has minutes from the beginning to the end

So we need to find the count of individuals that have the `Mets` column >= 3. We need to determine if the count is > 150 (after aggregating all the 7 days)

In [19]:
df.loc[df.Mets >= 3].Mets.count()

8

In [20]:
df.loc[(df['Mets']>= 1.6) & (df['Mets'] < 3)].Mets.count() #LIPA

215

In [21]:
df.loc[(df['Mets'] <= 1.5)].Mets.count() #Sedentary 1217

1217

Now we need to add Short-bout Long-bout MVPA where SB is 1 ~ 9 mins MVPA and LB is >= 10 mins MVPA. We need to sum the count of the Mets.

In [22]:
df.loc[df.Mets >= 3].shape[0]

8

The person above has no consecutive MVPA thus would be considered to have 8 mins of SB MVPA.

In [23]:
df.loc[df.Mets >= 3]

,Time,Mets,Timedif
527,2018-02-13 08:47:00,3.1,527.0
568,2018-02-13 09:28:00,3.1,568.0
691,2018-02-13 11:31:00,3.1,691.0
835,2018-02-13 13:55:00,3.0,835.0
838,2018-02-13 13:58:00,3.0,838.0
845,2018-02-13 14:05:00,3.1,845.0
846,2018-02-13 14:06:00,3.2,846.0
884,2018-02-13 14:44:00,3.0,884.0


In [24]:
(df.loc[df.Mets >= 3].iat[6,0] - df.loc[df.Mets >= 3].iat[5,0]).seconds > 60

False

In the above case, the person has had one consecutive MVPA minutes (i.e. 2 minutes) but otherwise they were all less than 9 minutes therefore these 8 minutes were all SB MVPA. The index number 845 and 846 indicates that time 845 and 846 were consecutive MVPA values.

We are going to use the `itertools` package `groupby` class to sort rows into consecutive times. First let's just convert the times into pure numbers so it's easier for us to count using the `itertools`

In [25]:
from itertools import groupby, accumulate
from operator import itemgetter, mul

The below code will iterate through the enumerated list of the time column with Mets >= 3. The groupby function will group the values which are the difference between the 0th and 1st index of the enumerated (i.e. 2-tuple) list. 

In [26]:
for k, g in groupby(enumerate(df.loc[df.Mets >= 3].Timedif.tolist()), lambda i: i[0] - i[1]):
    print(len(list(map(itemgetter(1), g))))

1
1
1
1
1
2
1


The below code on the other hand will do something similar but rather return a list of values that are number of consecutive minutes (i.e. consecutive 60 seconds with Mets >= 3).

In [27]:
# example list
data = [2, 3, 4, 5, 12, 13, 14, 15, 16, 17]
ranges = []
for key, group in groupby(enumerate(data), lambda i: i[0] - i[1]):
    group = list(map(itemgetter(1), group))
    ranges.append((group[-1]-group[0] + 1))
    
ranges

[4, 6]

The above code prints the list of consecutive times of MVPA. Now we will just create a function that will take a list and output a list of two values. The two values will be the sum of consecutive Long-bout and Short-bout MVPA times. For someone who had a MVPA value of >= 10 for two minute should have a value of 2 for the LB_MVPA value while if that same person had 20 minutes of MVPA values < 10, then that person should have a SB_MVPA value of 20 therefore the list would look like `[2, 20]` or some form that has two values.

In [28]:
def mvpa_sblb(list_data, thresh):
    """
    The input should be a list!
    This function returns a list of 2 values. 1 value is the number of short-bout MVPA 
    The other is the number of long-bout MVPA. 
    """
    
    #  first create empty list that will be our output 
    ranges = []

    # we need to iterate through the list. This for loop below will create a 
    # list of values that indicate the consecutive MVPA minutes. 
    # Therefore a number 2 in the list will indicate that
    # The participant had 2 minutes of consecutive time during the day(or week).
    # many thanks to https://stackoverflow.com/questions/2154249/identify-groups-of-continuous-numbers-in-a-list
    
    for key, group in groupby(enumerate(list_data), lambda i: i[0] - i[1]):
        group = list(map(itemgetter(1), group))
        ranges.append((group[-1]-group[0] + 1))
        
    # Now we need to check the list and sum the values that are > 9 which forms the value for
    # lb_mvpa. If values are <= 9 then they sum to be the value for sb_mvpa
    sb_list = []
    lb_list = []
    for i in ranges:
        if ( i >= thresh ):
            lb_list.append((i))
        else:
            sb_list.append((i))
    
    # sum the values in each list and return list
    finlist = [sum(sb_list),sum(lb_list)]
    
    return(finlist)

In [29]:
mvpa_sblb(data, thresh=10)

[10, 0]

Now try it on our data

In [30]:
mvpa_sblb(df.loc[df.Mets >= 3].Timedif, thresh = 10)

[8.0, 0]

We clearly see this person's total MVPA is composed of 'SB' only

Now lets try it after merging two files from the same person

In [31]:
import glob 
import pandas as pd
import re
path = r'../data' # path to csv files
# save the names of the files into a list
allFiles = ['../data/15kenoku20180211.csv', '../data/15kenoku20180212.csv']
frame = pd.DataFrame()
list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, encoding='cp932', skiprows= [0,1,2], header=0)
    df.columns = ["Time","Mets","random"]
    df.drop('random', axis=1, inplace=True)
    df["Time"] = (pd.Series([re.search('../data/\d*'+'kenoku'+'(\d+).csv', file_).group(1)]*df.shape[0]) + " " + df.Time)
    df['Time'] =  pd.to_datetime(df['Time'], format="%Y%m%d %H:%M:%S")
    df['Timedif'] = (df.Time - pd.DataFrame({"FTime": [df.Time.tolist()[0]]*df.shape[0]}).FTime).astype('timedelta64[m]')
    
    list_.append(df)
df = pd.concat(list_, ignore_index=True)

In [32]:
df.shape[0]

2880

In [33]:
df.head(2)

,Time,Mets,Timedif
0,2018-02-11 00:00:00,0.0,0.0
1,2018-02-11 00:01:00,0.0,1.0


In [34]:
mvpa_sblb(df.loc[df.Mets >= 3].Timedif, thresh = 10)

[16.0, 0]

In [35]:
finalout = pd.DataFrame({
        "SB_MVPA":[],
        "LB_MVPA":[],
        "Name": [],
    })
finalout.columns = ["SB_MVPA","LB_MVPA","Name"]
finalout.loc[0] = mvpa_sblb(df.loc[df.Mets >= 3].Timedif, thresh = 10) + ["kenoku"]
finalout

,SB_MVPA,LB_MVPA,Name
0,16.0,0.0,kenoku


## Merging multiple csv files row-wise

The following code is to retrieve files from the data folder and merge it to create a data frame as below:

In [36]:
import numpy as np

df_want = pd.DataFrame({
    "id": np.arange(10) + 1,
    "Time": [0, 1, 3, 5, 7, 8, 9, 10, 12, 13]
})
df_want.head()

,Time,id
0,0,1
1,1,2
2,3,3
3,5,4
4,7,5


Ideally, we would like to create a function that will take the identifying information from the `*.csv` files (e.g. 15kentaokuyama120210.csv, 15kentaokuyama120211.csv...) and load them in.

In [37]:
import glob
path = r'../data' # specify the path of the data files
allFiles = glob.glob(path + '/*.csv') # this creates a list of all the files in the folder with /*.csv extension

In [38]:
allFiles

['../data/15ericdong20180213.csv',
 '../data/15ericdong20180212.csv',
 '../data/forpractice20180213.csv',
 '../data/forpractice20180212.csv',
 '../data/15kenoku20180211.csv',
 '../data/forpractice-Copy5.csv',
 '../data/15ericdong20180211.csv',
 '../data/forpractice.csv',
 '../data/forpractice-Copy4.csv',
 '../data/15ericdong20180210.csv',
 '../data/15kenoku20180213.csv',
 '../data/forpractice20180211.csv',
 '../data/forpractice20180210.csv',
 '../data/15kenoku20180210.csv',
 '../data/15kenoku20180212.csv']

Now we want to specify a function that will take in the names that we want to use and extract the csv files that we will load in

In [39]:
def mvpa_create(name_list, csv_path):
    """
    This function takes a list of names and the data path in order to: 
    1. Merge the data frames row-wise. The individual's data will be stored in a pandas DataFrame
    column-wise.
    2. Then each individuals data will be used to calculate lb and sb mvpa 
    3. The output will be a data frame of 1 patient 1 row with columns representing mvpa statistics
    """
    
    #1. Merge data frames rowise
    import glob
    import pandas as pd
    import re
    allFiles = glob.glob(csv_path + "/*.csv")
    
    # save the individual data file names into key value pairs as dictionary
    filesdic = {}
    # iterate through name and append file to selectFiles
    for name in name_list:
        selectFiles = []
        for i in range(len(allFiles)):
            # since there will be errors if we don't bypass the error s from the
            # non working regexp list elements... we need a try/except statement here
            try:
                selectFiles.append(re.search('../data/\d*('+name+')(\d+).csv', allFiles[i]).group())
            except:
                pass
        filesdic[name] = selectFiles
    # to check the returned dictionary uncomment next line   
    #return(filesdic)
    
    # create finalout dataset that will be populated at the end
    finalout = pd.DataFrame({
        "SB_MVPA":[],
        "LB_MVPA":[],
        "Name": [],
    })
    finalout.columns = ["SB_MVPA","LB_MVPA","Name"]
    
    # here we iterate through each of the patient names
    for i in filesdic:
        frame = pd.DataFrame()
        list_ = []
        
        # we iterate through the files for each of the patients and create 1 dataframe
        for file_ in filesdic[i]:
            # read in the csv files onto an object `df`
            df = pd.read_csv(file_, index_col=None, encoding='cp932', skiprows = [0,1,2], header=0)
            df.columns = ["Time","Mets","random"] # names of column from japanese to english
            df.drop('random', axis=1, inplace=True) # drop last column
            
            # as we did before since the dates aren't added onto the time column we need to add the 
            # dates based on the csv file name
            df["Time"] = (pd.Series([re.search('../data/\d*'+i+'(\d+).csv', file_).group(1)]*df.shape[0]) + " " + df.Time)
            df['Time'] =  pd.to_datetime(df['Time'], format="%Y%m%d %H:%M:%S") # convert to datetime
            df['Timedif'] = (df.Time - pd.DataFrame({"FTime": [df.Time.tolist()[0]]*df.shape[0]}).FTime).astype('timedelta64[m]')
            list_.append(df)
        
        # Once the list_ file has been populated row-wise.. make it into pd dataframe
        frame = pd.concat(list_, ignore_index=True)
        # now use the mvpa_lbsb function to get the 2 value list
        finalout.loc[i] = mvpa_sblb(frame.loc[frame.Mets >= 3].Timedif, thresh = 10)+ [i]
    
    return(finalout)
    
tmp = mvpa_create(['kenoku','ericdong'], r'../data')

In [40]:
tmp

,SB_MVPA,LB_MVPA,Name
kenoku,32.0,0.0,kenoku
ericdong,32.0,0.0,ericdong


## Vuala!

The last out put is basically what we want in terms of the short-bout and long-bout MV physical activity. Other additional variable creation to come!